<a href="https://colab.research.google.com/github/daniel27a/missp/blob/main/pulp/Whiskas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Wykonał: 
##Daniel Sobczak 
###215342 7AiR2

Notebook zawiera rozwiązanie problemu liniowego przy pomocy narzędzia Pulp.

Poniższa komórka kodu zawiera instrukcję instalującą dodatek Pulp do Notebooka. 


In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

>Poniższy problem liniowy dotyczy minimalizacji kosztów produkcji puszki karmy dla kotów. Zmiennymi decyzyjnymi jest procentowa zawartość każdego ze składników w puszce. W badanym przypadku jest to kurczak, wołowina oraz sardynki. Każdy ze składników charakteryzuje się inną zawartością białka, tłuszczu, błonnika, soli oraz witaminy B3.

Składnik | Białko | Tłuszcz | Błonnik | Sól | Witamina B | Cena za gram
---|---|---|---|---|---|---
Kurczak | 0.1 | 0.08 | 0.001 | 0.002 | 0.003 | 0.013
Wołowina | 0.2 | 0.1 | 0.005 | 0.005 | 0.26 | 0.008
Sardynki | 0.25 | 0.11 | 0 | 0.00505 | 0.089 | 0.007


>Podstawowym ograniczeniem nałożonym na nasz produkt jest konieczność zapełnienia całej puszki, tj. suma procentowa wszystkich składników musi się równać 100%. Oprócz tego, w przypadku minimalizacji, musimy uwzględnić minimalną wartość każdej zmiennej czyli 0%.
  
>Kolejnymi ograniczeniami były minimalne oraz maksymalne zawartości mikro i makro składników na puszkę karmy. Koniecznym jest osiągnięcie minimalnej wymaganej zawartości wartościowych składników takich jak białko, tłuszcz i witamina oraz nieprzekroczenie pewnej zawartości błonnika oraz soli. 

>Dla uproszczenia przyjęto, że jeden gram produktu to 1% zawartości puszki.

>Znając cenę składników możemy przejść do rozwiązania problemu. Program zwraca procentową zawartość każdego ze składników oraz cenę wyprodukowania jednej puszki karmy. Informacja zostaje przekazana w postaci jednego zdania zawierającego wszystkie potrzebne informacje.

>Zmieniając cenę wpływamy na zawartość procentową poszczególnych składników.
W przypadku cen wpisanych w tabelę, skład produktu przedstawia wynik programu poniżej:
```
Status of problem: Optimal
Beef = 0.0 %
Chicken = 34.42623 %
Sardines = 65.57377 %
A can of product will cost 0.90655738 $ and will contain: 0.0% of Beef, 34.42623% of Chicken, 65.57377% of Sardines,
```




In [22]:
from pulp import *

# Name and type of problem
prob = LpProblem("The Whiskas' Cat Food Problem",LpMinimize)

# The 3 variables of Beef, Chicken and Sardines percentages are created with a lower limit of zero
x1=LpVariable("Chicken",0,)
x2=LpVariable("Beef",0)
x3=LpVariable("Sardines",0)

# The objective function is added to 'prob' first
prob += 0.013*x1 + 0.008*x2 + 0.007*x3, "Total Cost of Ingredients per can"

# The six constraints are added to the problem 
prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.25*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.11*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.00505*x3 <= 0.4, "SaltRequirement"
prob += 0.003*x1 + 0.026*x2 + 0.089*x3 >= 0.8, "VitaminBreq" 

prob.writeLP("WhiskasModel.lp")

# The solution of problem
prob.solve()
print("Status of problem:", LpStatus[prob.status])

# Listing of percentage content
for v in prob.variables():
    print(v.name, "=", v.varValue,'%')

# String to creating a summary sentence
percentages = ""

for v in prob.variables():
    percentages += str(v.varValue) + "% of " + v.name + ", "

# The summary sentence
print("A can of product will cost", value(prob.objective), "$ and will contain:", percentages)

print()

Status of problem: Optimal
Beef = 0.0 %
Chicken = 34.42623 %
Sardines = 65.57377 %
A can of product will cost 0.90655738 $ and will contain: 0.0% of Beef, 34.42623% of Chicken, 65.57377% of Sardines, 



/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
